In [2]:
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from pandas import DataFrame

In [3]:
#Time by Days
today = date.today()

In [4]:
#Total Jobs of Vietlam24h
url = "https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?hdn_tu_khoa=&tk_select_gate=&hdn_nganh_nghe_cap1=&hdn_dia_diem="
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

total_jobs_r = soup.find("span", class_="text-tim-nhat uppercase").get_text()
total_jobs = total_jobs_r[-5:-1]
print(total_jobs)

3838


In [5]:
#Original DataFrame
Vietlam24h = {'Time': today,
             'Total Jobs': total_jobs,
             }

master_df = DataFrame()

In [6]:
#Jobs by industry: lao dong pho thong
url_ind_list = ["https://vieclam24h.vn/mien-nam/viec-lam-lao-dong-pho-thong/viec-lam-theo-nganh-nghe",
               'https://vieclam24h.vn/mien-nam/viec-lam-quan-ly/viec-lam-theo-nganh-nghe',
               'https://vieclam24h.vn/mien-nam/viec-lam-chuyen-mon/viec-lam-theo-nganh-nghe',
               'https://vieclam24h.vn/mien-nam/viec-lam-ban-thoi-gian/viec-lam-theo-nganh-nghe']
result = []
for url_ind in url_ind_list:
    page = requests.get(url_ind)
    soup = BeautifulSoup(page.text, 'html.parser')
    soup.prettify()

    senority = soup.find('span', class_='txt-ghi mb10 font13 italic').get_text()
    senority = senority[16:]

    industries_url = []
    full_ind_dict = {}
    full_ind = []
    industries_jobs_no = []

    industries_url_raw = soup.find('div', class_='pl_8 pr_8').find_all('a', href=True)
    def unique(list1): 

        # insert the list to the set 
        list_set = set(list1) 
        # convert the set to the list 
        unique_list = (list(list_set)) 
        return unique_list

    industries_url_raw=unique(industries_url_raw)

    for industry in industries_url_raw:
        href = industry.get('href')
        industries_url.append(href)

    for industry in industries_url:
        page = requests.get(industry)

        soup = BeautifulSoup(page.text, 'html.parser')
        soup.prettify()

        industry_no_r = soup.find('div', class_='box_white mt_16 bold').get_text()
        industry_name_r = soup.find('div', class_='box_white mt_16 bold').find_all('span', class_='text_pink')[1].get_text()
        full_ind.append(industry_name_r)
        for s in industry_no_r.split(): 
            if s.isdigit():
                industries_jobs_no = int(s)

        full_ind_dict[f'{industry_name_r}'] = industries_jobs_no    
    result.append(full_ind_dict)

In [7]:
final = {}

def checkNone(x, key):
    if key not in x:
        return 0
    else:
        return x[key]

for key in result[3].keys():
    total = checkNone(result[0],key) + checkNone(result[1],key) + checkNone(result[2],key) + checkNone(result[3],key)
    final[key] = total

print(final) 
total_jobs = sum(final.values())


{'Xây dựng': 207, 'Công nghiệp': 76, 'Lao động phổ thông': 139, 'Tổ chức sự kiện-Quà tặng': 6, 'Thực tập': 2, 'Hàng gia dụng': 5, 'Nhân sự': 102, 'Thiết kế đồ hoạ web': 51, 'Bưu chính': 2, 'Hoá học-Sinh học': 61, 'Ngoại thương-Xuất nhập khẩu': 92, 'Thực phẩm-Đồ uống': 82, 'KD bất động sản': 47, 'Pháp lý-Luật': 23, 'Báo chí-Truyền hình': 13, 'Nông-Lâm-Ngư nghiệp': 22, 'Điện-Điện tử-Điện lạnh': 159, 'Kho vận-Vật tư': 104, 'Nhân viên kinh doanh': 513, 'Điện tử viễn thông': 32, 'Khách sạn-Nhà hàng': 72, 'Giáo dục-Đào tạo': 19, 'Tư vấn': 361, 'Du lịch': 13, 'Game': 1, 'Promotion Girl (PG)': 0, 'IT phần cứng/mạng': 59, 'In ấn-Xuất bản': 38, 'Vận tải': 62, 'Thủ công mỹ nghệ': 10, 'Dầu khí-Hóa chất': 20, 'Chứng khoán- Vàng': 2, 'Thương mại điện tử': 19, 'Thiết kế-Mỹ thuật': 83, 'Hàng không': 4, 'Dệt may - Da giày': 62, 'IT phần mềm': 77, 'Thời trang': 27, 'Ngành nghề khác': 46, 'Kế toán-Kiểm toán': 368, 'Ngân hàng': 76, 'Marketing-PR': 119, 'Hàng hải': 8, 'Thư ký-Trợ lý': 86, 'Hoạch định-Dự án

In [8]:
Vietlam24h = {'Time': today,
             'Total Jobs': total_jobs,
             'Industry': full_ind,
             }

df = DataFrame(Vietlam24h, columns=['Time', 'Total Jobs', 'Industry'])

In [9]:
senior_pd_r = pd.DataFrame.from_dict(result)
no_of_senior = senior_pd_r.sum(axis=1, skipna=True)
no_of_senior = pd.DataFrame(no_of_senior)

dictionary = no_of_senior.to_dict()
d = dictionary.get(0)
Vietlam24h.update(d)

df = DataFrame(Vietlam24h, columns=['Time', 'Total Jobs',3,0,2,1, 'Industry'])
df.rename(columns={3:'Intern', 0:'Junior', 2: 'Senior', 1: 'Manager/Director'}, inplace = True)

print(df)

          Time  Total Jobs  Intern  Junior  Senior  Manager/Director  \
0   2020-02-19        5622    49.0   431.0  4898.0             427.0   
1   2020-02-19        5622    49.0   431.0  4898.0             427.0   
2   2020-02-19        5622    49.0   431.0  4898.0             427.0   
3   2020-02-19        5622    49.0   431.0  4898.0             427.0   
4   2020-02-19        5622    49.0   431.0  4898.0             427.0   
..         ...         ...     ...     ...     ...               ...   
62  2020-02-19        5622    49.0   431.0  4898.0             427.0   
63  2020-02-19        5622    49.0   431.0  4898.0             427.0   
64  2020-02-19        5622    49.0   431.0  4898.0             427.0   
65  2020-02-19        5622    49.0   431.0  4898.0             427.0   
66  2020-02-19        5622    49.0   431.0  4898.0             427.0   

                    Industry  
0                   Xây dựng  
1                Công nghiệp  
2         Lao động phổ thông  
3   Tổ chức

In [14]:
senior_pd = senior_pd_r.transpose()

data = senior_pd.sum(axis = 1, skipna=True)
senior_pd.insert(4, 'No. of Jobs', data)
# senior_pd['Total'] = senior_pd.sum(axis = 1, skipna=True)
senior_pd = senior_pd.reset_index()
senior_pd.rename(columns = {'index':'Industry',0:'Junior',1: 'Manager/Director', 2:'Senior', 3: 'Intern'}, inplace = True)

print(senior_pd)

                Industry  Junior  Manager/Director  Senior  Intern  \
0     Sinh viên làm thêm     4.0               NaN     NaN    12.0   
1               Kỹ thuật    11.0              15.0   276.0     0.0   
2   Nhân viên kinh doanh    12.0               NaN   500.0     1.0   
3        Người giúp việc     3.0               NaN     NaN     0.0   
4         In ấn-Xuất bản     1.0               1.0    36.0     0.0   
..                   ...     ...               ...     ...     ...   
63     IT phần cứng/mạng     NaN               0.0    58.0     1.0   
64          Pháp lý-Luật     NaN               5.0    18.0     0.0   
65   Thiết kế đồ hoạ web     NaN               1.0    50.0     0.0   
66             Y tế-Dược     NaN               3.0    74.0     0.0   
67              Thực tập     NaN               NaN     NaN     2.0   

    No. of Jobs  
0          16.0  
1         302.0  
2         513.0  
3           3.0  
4          38.0  
..          ...  
63         59.0  
64         23.0

In [15]:
Vietlam24h_result = pd.merge(df,
                 senior_pd[['Industry','No. of Jobs']],
                 on='Industry')
# Vietlam24h_result.sort_values(by=['Total'])
print(Vietlam24h_result)

          Time  Total Jobs  Intern  Junior  Senior  Manager/Director  \
0   2020-02-19        5622    49.0   431.0  4898.0             427.0   
1   2020-02-19        5622    49.0   431.0  4898.0             427.0   
2   2020-02-19        5622    49.0   431.0  4898.0             427.0   
3   2020-02-19        5622    49.0   431.0  4898.0             427.0   
4   2020-02-19        5622    49.0   431.0  4898.0             427.0   
..         ...         ...     ...     ...     ...               ...   
62  2020-02-19        5622    49.0   431.0  4898.0             427.0   
63  2020-02-19        5622    49.0   431.0  4898.0             427.0   
64  2020-02-19        5622    49.0   431.0  4898.0             427.0   
65  2020-02-19        5622    49.0   431.0  4898.0             427.0   
66  2020-02-19        5622    49.0   431.0  4898.0             427.0   

                    Industry  No. of Jobs  
0                   Xây dựng        207.0  
1                Công nghiệp         76.0  
2  

In [ ]:
df.to_csv("vietlam24h.csv")